단어를 구성하는 글자를 features 로 생각하면 단어도 sparse vector 로 생각할 수 있습니다. Euclidean distance 는 글자의 order 가 없지만, edit distance 는 글자의 order 가 있는 vector 로 생각할 수도 있습니다. 그렇다면 inverted index 를 이용하여 edit distance 가 작은 단어 셋을 탐색할 수 있습니다.

이 튜토리얼에서 이용하는 패키지는 https://github.com/lovit/inverted_index_for_hangle_editdistance 에 있습니다.

In [1]:
import config
import fast_hangle_levenshtein

soynlp=0.0.49
added lovit_textmining_dataset


In [2]:
fast_hangle_levenshtein.__title__

'빠른 한글 수정 거리 검색을 위한 inverted index '

In [3]:
fast_hangle_levenshtein.__version__

'0.0.2'

## Toy example

총 8 개 단어를 indexer 에 입력하여 indexing 을 합니다. _index 에는 각 글자를 key 로, 해당 글자를 포함하는 글자들이 values 에 포함되어 있습니다.

In [4]:
from fast_hangle_levenshtein import LevenshteinIndex
indexer = LevenshteinIndex(verbose=True)

In [5]:
indexer.indexing('아이고 어이고 아이고야 아이고야야야야 어이구야 지화자 징화자 쟝화장'.split())

In [6]:
indexer._index

{'고': {'아이고', '아이고야', '아이고야야야야', '어이고'},
 '구': {'어이구야'},
 '아': {'아이고', '아이고야', '아이고야야야야'},
 '야': {'아이고야', '아이고야야야야', '어이구야'},
 '어': {'어이고', '어이구야'},
 '이': {'아이고', '아이고야', '아이고야야야야', '어이고', '어이구야'},
 '자': {'지화자', '징화자'},
 '장': {'쟝화장'},
 '쟝': {'쟝화장'},
 '지': {'지화자'},
 '징': {'징화자'},
 '화': {'쟝화장', '지화자', '징화자'}}

초/중/종성을 분리할 경우에는 각각 _cho_index, _jung_index, _jong_index 에 동일한 형식으로 인덱싱이 되어 있습니다.

In [7]:
indexer._cho_index

{'ㄱ': {'아이고', '아이고야', '아이고야야야야', '어이고', '어이구야'},
 'ㅇ': {'아이고', '아이고야', '아이고야야야야', '어이고', '어이구야'},
 'ㅈ': {'쟝화장', '지화자', '징화자'},
 'ㅎ': {'쟝화장', '지화자', '징화자'}}

verbose on 일 때에는 query term 에 포함된 글자를 지닌 글자들의 개수를 candidates 로 표시하고, max_distance 보다 작은 distance 를 지니는 후보들을 filtering 하여 그 개수를 표시합니다. `아`, `이`, `코` 중 한 개 이상의 글자를 포함하는 단어는 5 개가 있었으며, 그 중 2 개에 대해서만 실제 거리 계산을 합니다.

Levenshtein dsitance 는 오탈자 교정의 목적을 위해 이용되는 경우가 많은데, 오탈자는 한글자 혹은 두 글자 (초/중/종성 기준) 정도만 틀리기 때문에 max distance 를 충분히 작은 값으로 설정해도 됩니다.

In [8]:
indexer.verbose = True
indexer.levenshtein_search('아이코', max_distance=1)

query=아이코, candidates=5 -> 2, time=0.000602 sec.


[('아이고', 1)]

In [9]:
indexer.jamo_levenshtein_search('아이코')

query=아이코, candidates=8 -> 3, time=0.00137 sec.


[('아이고', 0.3333333333333333), ('어이고', 0.6666666666666666)]

## Financial text example

위의 git repository 에는 금융 관련 뉴스 분석용 13 만여개의 명사 사전이 있습니다. 이 사전을 로딩합니다.

In [10]:
with open('./inverted_index_for_hangle_editdistance/data/nouns_from_financial_news.json', encoding='utf-8') as f:
    import json    
    noun_scores = json.load(f)
len(noun_scores)

132864

In [11]:
list(noun_scores.keys())[:10]

['양식어가',
 '식품유통사',
 'ETN전담팀',
 '도로주행',
 '로우프라이스펀드',
 '국가브랜드',
 '대체부지',
 '한화솔라원',
 '박준영씨',
 '온라인마트']

132,864 개의 단어를 indexing 합니다.

In [12]:
financial_word_indexer = LevenshteinIndex(noun_scores)

character set 을 기준으로 모두 등장하는 글자를 찾기 때문에 순서는 달라질 수 있습니다. 이번에는 `분식회계`의 글자를 1 개 이상 포함하는 10,137 개의 단어 중 `max_distance = 1`  보다 작은 값을 지닌 7 개의 단어에 대해서만 거리 계산을 수행합니다.

In [13]:
financial_word_indexer.verbose = True
financial_word_indexer.levenshtein_search('분식회계')

query=분식회계, candidates=10137 -> 7, time=0.0122 sec.


[('분식회계', 0), ('분식회계설', 1), ('분식회', 1), ('분석회계', 1)]

In [14]:
financial_word_indexer.levenshtein_search('분식회계a')

query=분식회계a, candidates=10451 -> 3, time=0.016 sec.


[('분식회계설', 1), ('분식회계', 1)]

초/중/종성을 분리하는 jamo levenshtein distance 기준으로도 검색이 가능합니다.

In [15]:
financial_word_indexer.jamo_levenshtein_search('분식회계', max_distance=1)

query=분식회계, candidates=129099 -> 412, time=0.399 sec.


[('분식회계', 0),
 ('분석회계', 0.3333333333333333),
 ('부실회계', 0.6666666666666666),
 ('분석체계', 1.0),
 ('분식회계설', 1),
 ('분식회', 1)]

`max_distance` 를 작게 설정하면 실제 거리 계산을 하는 단어의 숫자도 줄어듭니다. 하지만 시간은 많이 줄어들지 않았습니다. Filtering 을 하는 overhead 가 무겁게 구현되어 있습니다.

In [16]:
financial_word_indexer.jamo_levenshtein_search('분식회계', max_distance=0.5)

query=분식회계, candidates=129099 -> 4, time=0.321 sec.


[('분식회계', 0), ('분석회계', 0.3333333333333333)]

## Compare times

Index 를 이용하는 경우와 그렇지 않은 경우의 시간을 비교합니다. 13 만여개의 단어와의 거리를 모두 계산하기 때문에 거리 계산의 시간이 걸립니다. 0.016 초에 할 수 있는 작업에 2.5 초가 걸렸습니다.

In [17]:
import time
from fast_hangle_levenshtein import levenshtein
from fast_hangle_levenshtein import jamo_levenshtein

query = '분식회계'

begin_time = time.time()
distance = {word:levenshtein(word, query) for word in noun_scores}
search_time = time.time() - begin_time

similars = sorted(filter(lambda x:x[1] <= 1, distance.items()), key=lambda x:x[1])
sorting_time = time.time() - begin_time

print('search time = {:.4} sec'.format(search_time))
print('sorting time = {:.4} sec'.format(sorting_time))
print(similars)

search time = 2.328 sec
sorting time = 2.346 sec
[('분식회계', 0), ('분식회', 1), ('분식회계설', 1), ('분석회계', 1)]


초/중/종성을 분리할 경우에는 스트링 연산을 하기 때문에 더 많이 느려집니다. 0.4 초 안에 해결되는 작업에 28 초가 걸립니다.

In [18]:
search_time = time.time()
distance = {word:jamo_levenshtein(word, query) for word in noun_scores}
search_time = time.time() - search_time
print('search time = {} sec'.format('%.2f'%search_time))

similars = sorted(filter(lambda x:x[1] <= 1, distance.items()), key=lambda x:x[1])
print(similars)

search time = 35.89 sec
[('분식회계', 0), ('분석회계', 0.3333333333333333), ('부실회계', 0.6666666666666666), ('분식회', 1), ('분석체계', 1.0), ('분식회계설', 1)]


이처럼 index 를 이용하여 불필요한 계산을 하지 않으면 빠른 nearest neighbor search 가 가능합니다. Edit distance 기준으로는 distance 가 작은 elements 를 효율적으로 찾을 수 있습니다.